In [1]:
from processing.processor import *
from analysis.analysis import *
import numpy as np
from scipy.spatial.distance import cdist, pdist
import seaborn as sns
from matplotlib import rcParams
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Data

In [2]:
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='rg', folder='data/processed/')

  2%|██▋                                                                                                               | 13/557 [00:00<00:04, 127.42it/s]

Reading files with generic numbers on receptors and gproteins.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [00:05<00:00, 98.77it/s]


In [3]:
ATOM_LIST=['CA']

In [4]:
f_act = p.make_filter(State='Active', Cl='Rhodopsin', gprotein=True)
# f_act = f_act[f_act['Method']=='cryo-EM']

In [5]:
len(f_act)

83

In [6]:
f_act[f_act['Family'].str.contains('q')]

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function
16,ACM1,Acetylcholine(muscarinic),A(Rhodopsin),Human,cryo-EM,6OIJ,3.3,R,Active,100,Gq/11,α11,Agonist
32,5HT2A,5-Hydroxytryptamine,A(Rhodopsin),Human,cryo-EM,6WHA,3.4,A,Active,100,Gq/11,αq,Agonist
71,HRH1,Histamine,A(Rhodopsin),Human,cryo-EM,7DFL,3.3,R,Active,100,Gq/11,αq,Agonist
76,CCKAR,Cholecystokinin,A(Rhodopsin),Human,cryo-EM,7MBY,2.4,R,Active,100,Gq/11,αq,Agonist


In [7]:
p.apply_filter(f_act)

In [8]:
len(p.dfl_list)

83

In [9]:
p.dfl_list.index('7MBY')

82

In [10]:
df = p.dfl[66]

In [11]:
df[df['gprot_pos'] != '']

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,identifier,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id
7016,7JJO,ATOM,A,C,19,15,GLU,7017,CA,C,...,ADRB1_MELGA,E,14,,0.0,0.0,,G.HN.32,E,E
7023,7JJO,ATOM,A,C,20,16,GLU,7024,CA,C,...,ADRB1_MELGA,E,15,,0.0,0.0,,G.HN.33,E,E
7030,7JJO,ATOM,A,C,21,17,LYS,7031,CA,C,...,ADRB1_MELGA,K,16,,0.0,0.0,,G.HN.34,K,K
7037,7JJO,ATOM,A,C,22,18,GLY,7038,CA,C,...,ADRB1_MELGA,G,17,,0.0,0.0,,G.HN.35,A,A
7044,7JJO,ATOM,A,C,23,19,GLN,7045,CA,C,...,ADRB1_MELGA,Q,18,,0.0,0.0,,G.HN.36,Q,Q
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10234,7JJO,ATOM,A,C,380,376,GLN,10235,CA,C,...,ADRB1_MELGA,Q,375,,0.0,0.0,,G.H5.08,F,F
10251,7JJO,ATOM,A,C,381,377,TYR,10252,CA,C,...,ADRB1_MELGA,Y,376,,0.0,0.0,,G.H5.09,N,N
10272,7JJO,ATOM,A,C,382,378,GLU,10273,CA,C,...,ADRB1_MELGA,E,377,,0.0,0.0,,G.H5.10,D,D
10287,7JJO,ATOM,A,C,383,379,LEU,10288,CA,C,...,ADRB1_MELGA,L,378,,0.0,0.0,,G.H5.11,C,C


In [12]:
eg = 47

In [13]:
p.table['angle'] = 0.000
for pdb in p.dfl_list: #p.dfl_list
    i = p.dfl_list.index(pdb)
    if pdb != p.dfl[i]['PDB'].iloc[0]:
        print("dfl list and df in dfl at corr index do not match!")
    if 'gen_pos' and 'gprot_pos' in list(p.dfl[i].columns):
        pass
    else:
        continue
        
    df_r = section_filter(p.dfl[i].copy(), chain='r', start=3.40, end=3.53)
    df_g = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=3, end=22)
    if len(df_r) > 0:
        if len(df_r[df_r['gen_pos'].str.contains('\?')])>30:
            df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
            pass
    xyz_r, mean_r = get_coords(df_r, False)
    xyz_g, mean_g = get_coords(df_g, False)
    
    if (xyz_r.shape[0]>6) & (xyz_g.shape[0]>6):
        v_r = get_helix(xyz_r-np.asarray(mean_r))
        v_g = get_helix(xyz_g-np.asarray(mean_g))
        vector_r = v_r * np.mgrid[-10:10:2j][:, np.newaxis]
        vector_g = v_g * np.mgrid[-10:10:2j][:, np.newaxis]

        angle_ = angle(v_r, v_g)
        if angle_ > 90:
            angle_ = 180 - angle_
        print(pdb, angle_, p.table[p.table['PDB']==pdb]['Family'].iloc[0])
        p.table.loc[p.table['PDB']==pdb, 'angle'] = angle_

3SN6 48.09079294551304 Gs
6CMO 24.526774181074018 Gi/o
6D9H 17.18481091509787 Gi/o
6DDE 29.82194044202106 Gi/o
6DDF 30.46671822807116 Gi/o
6K42 28.254114854331785 Gi/o
6KPF 25.233716412970267 Gi/o
6KPG 15.440945949437719 Gi/o
6LFM 26.97355004621977 Gi/o
6LFO 26.653155130395163 Gi/o
6LI3 34.497073639889656 Gs
6N4B 16.067286537638438 Gi/o
6OMM 24.853737811435025 Gi/o
6OS9 20.023563049814584 Gi/o
6OSA 14.913503343775716 Gi/o
6OY9 21.929037473632405 Gi/o
6OYA 18.46413932123872 Gi/o
6PT0 25.63917568643032 Gi/o
6QNO 23.12179889636198 Gi/o
6WHA 23.072922997378356 Gq/11
6WWZ 24.853236552985038 Gi/o
7BW0 40.383601542543744 Gs
7BZ2 28.4731085818768 Gs
7CFM 36.79816214765364 Gs
7CFN 35.747778506267 Gs
7CKW 32.346350393391276 Gs
7CKX 31.47572784481713 Gs
7CKY 32.021331306438356 Gs
7CKZ 29.169349205487322 Gs
7CRH 30.5958582937823 Gs
7CX2 33.86835595882263 Gs
7CX3 35.669544680463986 Gs
7CX4 34.8050878692824 Gs
7D7M 32.6166432254727 Gs
7DFL 32.581439588403725 Gq/11
7DHI 28.271513218674613 Gs
7DHR 29.

In [14]:
p.table[p.table['PDB']=='6OIK']

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function,angle
15,ACM2,Acetylcholine(muscarinic),A(Rhodopsin),Human,cryo-EM,6OIK,3.6,R,Active,100,Gi/o,αo,AgonistPAM,0.0


In [15]:
p.table[p.table['angle']>0].groupby('Family').count()

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Subtype,Function,angle
Family,,,,,,,,,,,,,
Gi/o,26,26,26,26,26,26,26,26,26,26,26,26,26
Gq/11,3,3,3,3,3,3,3,3,3,3,3,3,3
Gs,22,22,22,22,22,22,22,22,22,22,22,22,22


In [16]:
p.table[p.table['angle']==0]

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function,angle
0,5HT1B,5-Hydroxytryptamine,A(Rhodopsin),Human,cryo-EM,6G79,3.8,S,Active,100,Gi/o,αo,Agonist,0.0
3,DRD1,Dopamine,A(Rhodopsin),Human,cryo-EM,7JVQ,3.0,R,Active,100,Gs,αs,Agonist,0.0
6,ADA2B,Adrenoceptors,A(Rhodopsin),Human,cryo-EM,6K41,2.9,R,Active,100,Gi/o,αo,Agonist,0.0
15,ACM2,Acetylcholine(muscarinic),A(Rhodopsin),Human,cryo-EM,6OIK,3.6,R,Active,100,Gi/o,αo,AgonistPAM,0.0
16,ACM1,Acetylcholine(muscarinic),A(Rhodopsin),Human,cryo-EM,6OIJ,3.3,R,Active,100,Gq/11,α11,Agonist,0.0
19,5HT1E,5-Hydroxytryptamine,A(Rhodopsin),Human,cryo-EM,7E33,2.9,R,Active,100,Gi/o,αi1,Agonist,0.0
24,DRD3,Dopamine,A(Rhodopsin),Human,cryo-EM,7CMU,3.0,R,Active,100,Gi/o,αi1,Agonist,0.0
25,DRD3,Dopamine,A(Rhodopsin),Human,cryo-EM,7CMV,2.7,R,Active,100,Gi/o,αi1,Agonist,0.0
28,MC4R,Melanocortin,A(Rhodopsin),Human,cryo-EM,7AUE,3.0,R,Active,100,Gs,αs,Agonist,0.0
29,AA2AR,Adenosine,A(Rhodopsin),Human,X-ray,5G53,3.4,A,Active,100,Gs,αs,Agonist,0.0


In [17]:
p.table[p.table['angle']>0].groupby('Family').mean()['angle']

Family
Gi/o     22.841702
Gq/11    27.985651
Gs       33.895601
Name: angle, dtype: float64

In [18]:
# angle between last 26-11:26-4  and 3-15 

In [19]:
small = p.table[['PDB', 'uniprot(gene)', 'Family', 'angle']].copy()
small[(p.table['angle']>0)&(small['Family']=='Gi/o')].sort_values('uniprot(gene)').reset_index(drop=True)#.to_csv('gio_angle_tm3_h5.csv')

,PDB,uniprot(gene),Family,angle
0,7E2X,5HT1A,Gi/o,24.405260
1,7E2Z,5HT1A,Gi/o,23.350048
2,7E2Y,5HT1A,Gi/o,23.383962
3,7E32,5HT1D,Gi/o,21.286536
4,6D9H,AA1R,Gi/o,17.184811
5,6K42,ADA2B,Gi/o,28.254115
6,6WWZ,CCR6,Gi/o,24.853237
7,6N4B,CNR1,Gi/o,16.067287
8,6KPG,CNR1,Gi/o,15.440946
9,6KPF,CNR2,Gi/o,25.233716


In [20]:
small[(p.table['angle']>0)&(small['Family']=='Gs')].sort_values('uniprot(gene)').reset_index(drop=True)

,PDB,uniprot(gene),Family,angle
0,7JJO,ADRB1,Gs,38.358475
1,7DHR,ADRB2,Gs,29.670930
2,7DHI,ADRB2,Gs,28.271513
3,7BZ2,ADRB2,Gs,28.473109
4,3SN6,ADRB2,Gs,48.090793
5,7MBX,CCKAR,Gs,36.208022
6,7CKY,DRD1,Gs,32.021331
7,7CRH,DRD1,Gs,30.595858
8,7CKW,DRD1,Gs,32.346350
9,7CKZ,DRD1,Gs,29.169349


In [21]:
small[(p.table['angle']>0)&(small['Family']=='Gq/11')].sort_values('uniprot(gene)').reset_index(drop=True)

,PDB,uniprot(gene),Family,angle
0,6WHA,5HT2A,Gq/11,23.072923
1,7MBY,CCKAR,Gq/11,28.302592
2,7DFL,HRH1,Gq/11,32.581440


In [22]:
df = p.dfl[p.dfl_list.index('7CFM')]

In [23]:
df[df['gprot_pos'].str.contains('HN')]

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,identifier,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id
1,7CFM,ATOM,A,A,12,12,GLN,2,CA,C,...,GPBAR_HUMAN,Q,11,,0.0,0.0,,G.HN.29,Q,Q
10,7CFM,ATOM,A,A,13,13,ARG,11,CA,C,...,GPBAR_HUMAN,R,12,,0.0,0.0,,G.HN.30,R,R
21,7CFM,ATOM,A,A,14,14,ASN,22,CA,C,...,GPBAR_HUMAN,N,13,,0.0,0.0,,G.HN.31,N,N
29,7CFM,ATOM,A,A,15,15,GLU,30,CA,C,...,GPBAR_HUMAN,E,14,,0.0,0.0,,G.HN.32,E,E
38,7CFM,ATOM,A,A,16,16,GLU,39,CA,C,...,GPBAR_HUMAN,E,15,,0.0,0.0,,G.HN.33,E,E
47,7CFM,ATOM,A,A,17,17,LYS,48,CA,C,...,GPBAR_HUMAN,K,16,,0.0,0.0,,G.HN.34,K,K
56,7CFM,ATOM,A,A,18,18,ALA,57,CA,C,...,GPBAR_HUMAN,A,17,,0.0,0.0,,G.HN.35,A,A
61,7CFM,ATOM,A,A,19,19,GLN,62,CA,C,...,GPBAR_HUMAN,Q,18,,0.0,0.0,,G.HN.36,Q,Q
70,7CFM,ATOM,A,A,20,20,ARG,71,CA,C,...,GPBAR_HUMAN,R,19,,0.0,0.0,,G.HN.37,R,R
81,7CFM,ATOM,A,A,21,21,GLU,82,CA,C,...,GPBAR_HUMAN,E,20,,0.0,0.0,,G.HN.38,E,E


In [24]:
df[(df['gen_pos1']>=8.5)&(df['gen_pos1']<=9)&(df['label_atom_id']=='CA')]

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,identifier,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id
7912,7CFM,ATOM,R,E,287,287,TYR,7913,CA,C,...,GPBAR_HUMAN,Y,287,8.50x50,8.50,50.0,Y,,,
7924,7CFM,ATOM,R,E,288,288,THR,7925,CA,C,...,GPBAR_HUMAN,T,288,8.51x51,8.51,51.0,T,,,
7931,7CFM,ATOM,R,E,289,289,ALA,7932,CA,C,...,GPBAR_HUMAN,A,289,8.52x52,8.52,52.0,A,,,
7936,7CFM,ATOM,R,E,290,290,PRO,7937,CA,C,...,GPBAR_HUMAN,P,290,8.53x53,8.53,53.0,P,,,
7943,7CFM,ATOM,R,E,291,291,TRP,7944,CA,C,...,GPBAR_HUMAN,W,291,8.54x54,8.54,54.0,W,,,
7957,7CFM,ATOM,R,E,292,292,ARG,7958,CA,C,...,GPBAR_HUMAN,R,292,8.55x55,8.55,55.0,R,,,


In [25]:
small[small['Family']=='Gs'].reset_index(drop=True).sort_values('uniprot(gene)') #.to_csv('gs_angle_tm3_h5.csv')

,PDB,uniprot(gene),Family,angle
16,6GDG,AA2AR,Gs,0.000000
10,5G53,AA2AR,Gs,0.000000
23,7JJO,ADRB1,Gs,38.358475
29,6NI3,ADRB2,Gs,0.000000
28,7DHR,ADRB2,Gs,29.670930
27,7DHI,ADRB2,Gs,28.271513
26,7BZ2,ADRB2,Gs,28.473109
22,6E67,ADRB2,Gs,0.000000
14,3SN6,ADRB2,Gs,48.090793
32,7MBX,CCKAR,Gs,36.208022


In [26]:
small[small['Family']=='Gq/11'].reset_index(drop=True).sort_values('uniprot(gene)')#.to_csv('gq_angle_tm3_h5.csv')

,PDB,uniprot(gene),Family,angle
1,6WHA,5HT2A,Gq/11,23.072923
0,6OIJ,ACM1,Gq/11,0.000000
3,7MBY,CCKAR,Gq/11,28.302592
2,7DFL,HRH1,Gq/11,32.581440


In [33]:
pdb = '6N4B'
i = p.dfl_list.index(pdb)
if pdb != p.dfl[i]['PDB'].iloc[0]:
    print("dfl list and df in dfl at corr index do not match!")

df_rh8 = section_filter(p.dfl[i].copy(), chain='r', start=8.50, end=8.60)
df_rtm7 = section_filter(p.dfl[i].copy(), chain='r', start=7.40, end=7.53)
df_g = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=1, end=27)
if len(df_r) > 0:
    if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
        df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
        pass
xyz_r1, mean_r1 = get_coords(df_rh8, False)
xyz_r2, mean_r2 = get_coords(df_rtm7, False)

print(xyz_r1.shape)
print(xyz_r2.shape)
xyz_g, mean_g = get_coords(df_g, False)

if (xyz_r1.shape[0]>6) & (xyz_r2.shape[0]>6) & (xyz_g.shape[0]>6):
    v_r1 = get_helix(xyz_r1-np.asarray(mean_r1))
    v_r2 = get_helix(xyz_r2-np.asarray(mean_r2))
    v_g = get_helix(xyz_g-np.asarray(mean_g))
    vector_r1 = v_r1 * np.mgrid[-10:10:2j][:, np.newaxis]
    vector_r2 = v_r2 * np.mgrid[-10:10:2j][:, np.newaxis]
    vector_g = v_g * np.mgrid[-10:10:2j][:, np.newaxis]

    angle_ = angle(v_r1, v_g)
    if angle_ < 90:
        angle_ = 180 - angle_
print(angle_)

(8, 3)
(14, 3)
116.37136370604891


In [34]:
vector_r1

array([[ 6.38172066,  7.39074427, -2.1565112 ],
       [-6.38172066, -7.39074427,  2.1565112 ]])

# Plotting helices

In [35]:
plot_helices([[xyz_r1, vector_r1, mean_r1, 'Receptor'],
              [xyz_r2, vector_r2, mean_r2, 'Receptor'], 
              [xyz_g, vector_g, mean_g, 'Gprotein']])

In [ ]:
plot_helices([[xyz_r, vector_r, mean_r, 'Receptor'], [xyz_g, vector_g, mean_g, 'Gprotein']]) # 	7E2X

In [ ]:
plot_helices([[xyz_r, vector_r, mean_r, 'Receptor'], [xyz_g, vector_g, mean_g, 'Gprotein']]) # 	7E2Y

In [ ]:
# full interaction table

In [ ]:
pdb = '6N4B'
i = p.dfl_list.index(pdb)
if pdb != p.dfl[i]['PDB'].iloc[0]:
    print("dfl list and df in dfl at corr index do not match!")

df_r = section_filter(p.dfl[i].copy(), chain='r', start=3.40, end=3.53)
df_g = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=3, end=22)
if len(df_r) > 0:
    if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
        df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
        pass
xyz_r, mean_r = get_coords(df_r, False)
xyz_g, mean_g = get_coords(df_g, False)

if (xyz_r.shape[0]>6) & (xyz_g.shape[0]>6):
    v_r = get_helix(xyz_r-np.asarray(mean_r))
    v_g = get_helix(xyz_g-np.asarray(mean_g))
    vector_r = v_r * np.mgrid[-10:10:2j][:, np.newaxis]
    vector_g = v_g * np.mgrid[-10:10:2j][:, np.newaxis]

    angle_ = angle(v_r, v_g)
    if angle_ < 90:
        angle_ = 180 - angle_

# Full Helices Table

In [ ]:
G_SECTION_DICT = {}
R_SECTION_DICT = {'TM1': (1.45, 1.55),
                  'TM2': (2.45, 2.55),
                  'TM3': (3.40, 3.53),
                  'TM4': (4.45, 4.55),
                  'TM5': (5.45, 5.55),
                  'TM6': (6.45, 6.55),
                  'TM7': (7.4, 7.53),
                  'H8':  (8.5, 8.6),}

In [ ]:
def count_g_positions(data):
    cols = [x for x in range(100)]
    gs_count_df = pd.DataFrame(columns=cols)
    g_section_list = list(G_SECTION_DICT.keys())
    for i in range(len(data)):
        df = data[i]
        if 'gprot_pos' in list(df.columns):
            new_g_section_list = list(set([x.split('.')[1] for x in df.gprot_pos.unique() if len(x.split('.'))==3]))
            g_section_list += new_g_section_list
            for gs in new_g_section_list:
                if gs not in list(gs_count_df.index):
                    gs_count_df.loc[gs,:] = 0
                posis = [int(x.split('.')[-1]) for x in df.gprot_pos.unique() if (gs in x) & (len(x.split('.'))==3)]
                for pos in posis:
                    gs_count_df.loc[gs, pos] += 1
    return gs_count_df.sort_index()

In [ ]:
def find_cont_sections_g(gs_count_df, min_count=40, min_length=8):
    def find_sections(nums, min_length):
        nums.sort()
        nums.append(1e9)
        ans=[]
        l=nums[0]
        for i in range(1,len(nums)):
            if nums[i] != nums[i-1] + 1:
                if (nums[i-1] - l) >= min_length:
                    ans.append((l, nums[i-1]))
                l=nums[i]
        return ans
    g_section_dict = {}
    for section in list(gs_count_df.index):
        sec = gs_count_df.loc[section, (gs_count_df > 0).any(axis=0)]
        sec = sec.to_dict()
        sec_keys = [x for x in list(sec.keys()) if sec[x] >= min_count]
        g_section_dict[section] = find_sections(sec_keys, min_length)
    return g_section_dict

In [ ]:
def make_cont_section_dict_g(cont_sec_g):
    cont_sec_g_dict = {}
    for key in list(cont_sec_g.keys()):
        sections = cont_sec_g[key]
        for s, sec in enumerate(sections):
            cont_sec_g_dict[key+'_'+str(s)] = sec
    return cont_sec_g_dict

In [ ]:
gs_count_df = count_g_positions(p.dfl)

In [ ]:
gs_count_df

In [ ]:
# list(set(list(g_section_list)))

In [ ]:
cont_sec_g = find_cont_sections_g(gs_count_df, min_count=30, min_length=6)

In [ ]:
G_SECTION_DICT = make_cont_section_dict_g(cont_sec_g)

In [ ]:
G_SECTION_DICT['H5_1'] = (13, 23)
G_SECTION_DICT['H4_0'] = (3, 15)

In [ ]:
G_SECTION_DICT

In [ ]:
def calculate_section_helices(data):
    g_section_list = list(G_SECTION_DICT.keys())
    r_section_list = list(R_SECTION_DICT.keys())
    columns = g_section_list + r_section_list
    section_diff_df = pd.DataFrame(columns=columns)
    for i in range(len(data)):
        pdb = data[i].iloc[0]['PDB']
        if 'gen_pos' and 'gprot_pos' in list(data[i].columns):
            for c in columns:
                if c in r_section_list:
                    df_r = section_filter(data[i].copy(), chain='r', start=R_SECTION_DICT[c][0], end=R_SECTION_DICT[c][1])
                    if len(df_r) > 0:
                        if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
                            df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
                    xyz_r, mean_r = get_coords(df_r, False)
                    if (xyz_r.shape[0]>6):
                        v_r = get_helix(xyz_r-np.asarray(mean_r))
                        section_diff_df.loc[pdb, c] = v_r
                elif (c in g_section_list):
                    c_, c_idx = c.split('_')
                    if c_ in [x.split('.')[1] for x in list(data[i].gprot_pos.unique()) if len(x.split('.'))==3]:
                        df_g = section_filter(data[i].copy(), chain='g', gprot_region=c_, 
                                              start=G_SECTION_DICT[c][0], end=G_SECTION_DICT[c][1])
                        xyz_g, mean_g = get_coords(df_g, False)
                        if (xyz_g.shape[0]>6):
                            v_g = get_helix(xyz_g-np.asarray(mean_g))
                            section_diff_df.loc[pdb, c] = v_g
    return section_diff_df

In [ ]:
p.dfl[0]

In [ ]:
R_SECTION_DICT['TM6']

In [ ]:
section_helices = calculate_section_helices(p.dfl)

In [ ]:
section_helices.reset_index()['H4_0'].iloc[2]

In [ ]:
section_helices.loc['3SN6', 'H1_0']

In [ ]:
rois = list(section_helices.columns)

In [ ]:
def calc_angles_between_helices(section_helices, idx):
    cols = list(section_helices.columns)
    section_angles_df = pd.DataFrame(columns=cols)
    for i, c1 in enumerate(cols):
        for j, c2 in enumerate(cols):
            if i != j:
                section_helix1 = section_helices[c1].iloc[idx]
                section_helix2 = section_helices[c2].iloc[idx]
                if not (isinstance(section_helix1, float) | isinstance(section_helix2, float)):
                    angle_ = angle(section_helix1, section_helix2)
                    if angle_ > 90:
                        angle_ = 180 - angle_
                    section_angles_df.loc[c1, c2] = angle_
                else:
                    section_angles_df.loc[c1, c2] = np.nan
            else:
                section_angles_df.loc[c1, c2] = 0.000000
    return section_angles_df.T

In [ ]:
calc_angles_between_helices(section_helices, 2)

In [ ]:
ang_helices = []
for i in range(len(section_helices)):
    ang_helices.append(calc_angles_between_helices(section_helices, i))

In [ ]:
std_df = pd.DataFrame(columns=rois)
mean_df = pd.DataFrame(columns=rois)
for r1, roi1 in enumerate(rois):
    for r2, roi2 in enumerate(rois):
        l_r1r2 = []
        for df in ang_helices:
            cell = df.loc[roi1, roi2]
            if cell > 0:
                l_r1r2.append(cell)
        if len(l_r1r2) > 0:
            mean_r1r2 = np.mean(l_r1r2)
            std_r1r2 = np.std(l_r1r2)
            std_df.loc[roi1, roi2] = std_r1r2
            mean_df.loc[roi1, roi2] = mean_r1r2

# Plotting STD for helix angles

In [ ]:
std_df

In [ ]:
std_df_interaction = std_df.loc[list(G_SECTION_DICT.keys()), list(R_SECTION_DICT)].copy()

In [ ]:
std_df_gprot = std_df.loc[list(G_SECTION_DICT.keys()), list(G_SECTION_DICT)].copy()

In [ ]:
std_df_receptor = std_df.loc[[x for x in list(R_SECTION_DICT.keys()) if x!='H8'],list(R_SECTION_DICT.keys())].copy()

In [ ]:
name = 'Standard deviation in helices directional angles'
rcParams['figure.figsize'] = 20, 15
low_section = 50
high_section = 180
dense_map = std_df_interaction[(std_df_interaction < low_section) | (std_df_interaction > high_section)]\
    .dropna(axis=0, how='all').dropna(axis=1, how='all')
ax = sns.heatmap(dense_map.astype(float), cmap='RdYlGn_r', linewidths=.1)
ax.set_title(name)
ax.plot()

In [ ]:
name = 'Standard deviation in helices directional angles'
rcParams['figure.figsize'] = 20, 15
low_section = 20
high_section = 60
dense_map = std_df_gprot[(std_df_gprot < low_section) | (std_df_gprot > high_section)]\
    .dropna(axis=0, how='all').dropna(axis=1, how='all')
ax = sns.heatmap(dense_map.astype(float), cmap='RdYlGn_r', linewidths=.1)
ax.set_title(name)
ax.plot()

In [ ]:
name = 'Standard deviation in helices directional angles'
rcParams['figure.figsize'] = 20, 15
low_section = 50
high_section = 100
dense_map = std_df_receptor[(std_df_receptor < low_section) | (std_df_receptor > high_section)]\
    .dropna(axis=0, how='all').dropna(axis=1, how='all')
ax = sns.heatmap(dense_map.astype(float), cmap='RdYlGn_r', linewidths=.1)
ax.set_title(name)
ax.plot()

In [ ]:
name = 'Standard deviation in helices directional angles'
rcParams['figure.figsize'] = 20, 15
low_section = 6
high_section = 180
dense_map = std_df[(std_df < low_section) | (std_df > high_section)]\
    .dropna(axis=0, how='all').dropna(axis=1, how='all')
ax = sns.heatmap(dense_map.astype(float), cmap='RdYlGn_r', linewidths=.1)
ax.set_title(name)
ax.plot()

In [ ]:
name = 'Mean in helices directional angles'
rcParams['figure.figsize'] = 20, 15
ax = sns.heatmap(mean_df.astype(float), cmap='RdYlGn_r', linewidths=.1)
ax.set_title(name)
ax.plot()

# H5, HN

In [ ]:
p.table['angle'] = 0.000
for pdb in p.dfl_list: #p.dfl_list
    i = p.dfl_list.index(pdb)
    if 'gprot_pos' in list(p.dfl[i]):
        if pdb != p.dfl[i]['PDB'].iloc[0]:
            print("dfl list and df in dfl at corr index do not match!")

        df_g1 = section_filter(p.dfl[i].copy(), chain='g', gprot_region='HN', start=30, end=53)
        df_g2 = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=3, end=23)

        if len(df_r) > 0:
            if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
                df_r = df_r[~df_r['gen_pos'].str.contains('\?')]

        xyz_1, mean_1 = get_coords(df_g1, False)
        xyz_2, mean_2 = get_coords(df_g2, False)

        if (xyz_1.shape[0]>6) & (xyz_2.shape[0]>6):
            v_1 = get_helix(xyz_1-np.asarray(mean_1))  # UNDIRECTED
            v_2 = get_helix(xyz_2-np.asarray(mean_2))
            angle_ = angle(v_1, v_2)
            if angle_ > 90:
                angle_ = 180 - angle_
            print(pdb, angle_, p.table[p.table['PDB']==pdb]['Family'].iloc[0])
            p.table.loc[p.table['PDB']==pdb, 'angle'] = angle_

In [ ]:
small = p.table[['PDB', 'uniprot(gene)', 'Family', 'angle']].copy()
small[(p.table['angle']>0)&(small['Family']=='Gi/o')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gio_angle_hn_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gs')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gs_angle_hn_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gq/11')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gs_angle_hn_h5.csv')

In [ ]:
p.table[p.table['angle']>0].groupby('Family').std()['angle']

# H5 - H5 

In [ ]:
p.table['angle'] = 0.000
for pdb in p.dfl_list: #p.dfl_list
    i = p.dfl_list.index(pdb)
    if 'gprot_pos' in list(p.dfl[i]):
        if pdb != p.dfl[i]['PDB'].iloc[0]:
            print("dfl list and df in dfl at corr index do not match!")

        df_g1 = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=3, end=13)
        df_g2 = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=13, end=24)
        if len(df_r) > 0:
            if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
                df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
                pass
        xyz_1, mean_1 = get_coords(df_g1, False)
        xyz_2, mean_2 = get_coords(df_g2, False)

        if (xyz_1.shape[0]>6) & (xyz_2.shape[0]>6):
            v_1 = get_helix(xyz_1-np.asarray(mean_1))  # UNDIRECTED
            v_2 = get_helix(xyz_2-np.asarray(mean_2))
            angle_ = angle(v_1, v_2)
            if angle_ > 90:
                angle_ = 180 - angle_
            print(pdb, angle_, p.table[p.table['PDB']==pdb]['Family'].iloc[0])
            p.table.loc[p.table['PDB']==pdb, 'angle'] = angle_

In [ ]:
small = p.table[['PDB', 'uniprot(gene)', 'Family', 'angle']].copy()
small[(p.table['angle']>0)&(small['Family']=='Gi/o')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gio_angle_h5_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gs')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gs_angle_h5_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gq/11')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gq11_angle_h5_h5.csv')

In [ ]:
p.table[p.table['angle']>0].groupby('Family').std()['angle']

# TM3 H5

In [ ]:
p.table['angle'] = 0.000
for pdb in p.dfl_list: #p.dfl_list
    if 'gprot_pos' in list(p.dfl[i].columns):
        i = p.dfl_list.index(pdb)
        if pdb != p.dfl[i]['PDB'].iloc[0]:
            print("dfl list and df in dfl at corr index do not match!")

        df_r = section_filter(p.dfl[i].copy(), chain='r', start=3.40, end=3.53)
        df_g = section_filter(p.dfl[i].copy(), chain='g', gprot_region='H5', start=3, end=22)
        if len(df_r) > 0:
            if len(df_r[df_r['gen_pos'].str.contains('\?')])>3:
                df_r = df_r[~df_r['gen_pos'].str.contains('\?')]
                pass
        xyz_r, mean_r = get_coords(df_r, False)
        xyz_g, mean_g = get_coords(df_g, False)

        if (xyz_r.shape[0]>6) & (xyz_g.shape[0]>6):
            v_r = get_helix(xyz_r-np.asarray(mean_r))
            v_g = get_helix(xyz_g-np.asarray(mean_g))
            vector_r = v_r * np.mgrid[-10:10:2j][:, np.newaxis]
            vector_g = v_g * np.mgrid[-10:10:2j][:, np.newaxis]

            angle_ = angle(v_r, v_g)
            if angle_ > 90:
                angle_ = 180 - angle_
            print(pdb, angle_, p.table[p.table['PDB']==pdb]['Family'].iloc[0])
            p.table.loc[p.table['PDB']==pdb, 'angle'] = angle_

In [ ]:
small = p.table[['PDB', 'uniprot(gene)', 'Family', 'angle']].copy()
small[(p.table['angle']>0)&(small['Family']=='Gi/o')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gio_angle_tm3_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gs')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gio_angle_tm3_h5.csv')

In [ ]:
small[(p.table['angle']>0)&(small['Family']=='Gq/11')].sort_values('uniprot(gene)').reset_index(drop=True)\
    .to_csv('gq11_angle_tm3_h5.csv')

In [ ]:
p.table[p.table['angle']>0].groupby('Family').std()['angle']

# Comparing families

Look at differences in STDs/Means in angles and distances (or Hbonds etc)!

# Cluster similar groups

Do (unsupervised) clusterings resemble the families? What subgroups within families could we find?